In [32]:
# Data Manipulation and Analysis
import pandas as pd  # For data manipulation and analysis
import numpy as np  # For numerical operations

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report  # For model evaluation metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler  # For scaling features


# Statistical Analysis
import scipy.stats as stats  # For statistical functions
from sklearn.impute import SimpleImputer  # For missing value imputation
from sklearn.linear_model import Ridge, Lasso, ElasticNet, BayesianRidge  # For regression models
from sklearn.tree import DecisionTreeRegressor  # For decision tree models
from xgboost import XGBClassifier  # For XGBoost classification model

# Handling Missing Data
from sklearn.experimental import enable_iterative_imputer  # Enables IterativeImputer
from sklearn.impute import SimpleImputer, IterativeImputer  # For missing value imputation

# Model Selection and Evaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report  # For model evaluation metrics

# Handling Imbalanced Datasets
from imblearn.under_sampling import RandomUnderSampler, TomekLinks  # For under-sampling techniques
from sklearn.metrics import roc_curve, auc

## Code

In [2]:
def drop_columns(df, columns_to_drop=['ID', 'Column9']):
    df.drop(columns=columns_to_drop, inplace=True)
    return df

In [3]:
def categorize_columns(df):
    
    col_names = df.columns.tolist()
    cate_col = []
    continue_col = []

    for i in col_names:
        value_count = df[i].value_counts()
        arr = list(value_count.index)
        arr.sort()

        is_continuous = True

        for j in range(len(arr) - 1):
            if arr[j + 1] != arr[j] + 1:
                is_continuous = False
                break

        if is_continuous:
            cate_col.append(i)
        else:
            continue_col.append(i)

    return cate_col, continue_col 

In [4]:
def fillna_with_mode(df, column='Column0'):
    col_mode = stats.mode(df[[column]]).mode[0]
    df[column] = df[column].fillna(col_mode)
    return df

In [5]:
def impute_missing_with_median(df, columns=['Column8', 'Column15', 'Column6']):
    
    median_imputer = SimpleImputer(strategy='median')
    
    for column in columns:
        df[[column]] = median_imputer.fit_transform(df[[column]])
    
    return df

In [6]:
def combined_imputation(df, continue_col):
    # Define the estimators to use for imputation
    regression_estimators = [Ridge(), Lasso(), ElasticNet(), BayesianRidge(), DecisionTreeRegressor()]

    # Create a dictionary to store imputed tables
    tables = {}

    # Extract the continuous columns data
    x = df[continue_col].copy()

    # Loop over each estimator to perform imputation
    for estimator in regression_estimators:
        print(f'Estimating with: {estimator.__class__.__name__}')
        
        # Perform iterative imputation
        imputer = IterativeImputer(estimator=estimator)
        df_imputed = imputer.fit_transform(x)
        df_imputed = pd.DataFrame(df_imputed, columns=x.columns)

        # Store imputed data in the tables dictionary
        tables[estimator.__class__.__name__] = df_imputed

        # Display variance differences
        for i in continue_col:
            original_variance = df[i].var()
            imputed_variance = df_imputed[i].var()
            variance_difference = original_variance - imputed_variance
            print(f'Feature: {i}, Variance Difference: {variance_difference}')
        
        print('\n')

    # Apply specific imputed columns back to the original DataFrame
    for estimator_name, table in tables.items():
        if estimator_name == 'Lasso':
            df[['Column3', 'Column4']] = table[['Column3', 'Column4']]
        elif estimator_name == 'Ridge':
            df['Column14'] = table['Column14']
        elif estimator_name == 'DecisionTreeRegressor':
            df['Column5'] = table['Column5']  # You can extend for more columns

    return df

In [7]:
def remove_outliers(df):
    
    iqr_columns = ['Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column8']
    percentile_columns = ['Column6', 'Column7', 'Column14', 'Column15', 'Column17', 'Column18']

    lower_percentile = 0.01
    upper_percentile = 0.99
    
    # Create a copy of the DataFrame to avoid modifying the original
    df_cleaned = df.copy()

    # Remove outliers using the IQR method
    for col in iqr_columns:
        if col in df_cleaned.columns and df_cleaned[col].nunique() > 1:  # Check if column exists and has variance
            Q1 = df_cleaned[col].quantile(0.25)
            Q3 = df_cleaned[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            df_cleaned = df_cleaned[(df_cleaned[col] >= lower_bound) & (df_cleaned[col] <= upper_bound)]
    
    # Remove outliers using the percentile method
    for col in percentile_columns:
        if col in df_cleaned.columns and df_cleaned[col].nunique() > 1:  # Check if column exists and has variance
            lower_bound = df_cleaned[col].quantile(lower_percentile)
            upper_bound = df_cleaned[col].quantile(upper_percentile)
            
            df_cleaned = df_cleaned[(df_cleaned[col] >= lower_bound) & (df_cleaned[col] <= upper_bound)]
    
    return df_cleaned

In [8]:
def apply_tomek_links(df):

    # Check if the target column exists
    if 'target' not in df.columns:
        raise ValueError("The DataFrame must contain a 'target' column.")
    
    # Separate features and target
    X = df.drop(['target'], axis='columns')
    y = df['target']
    
    # Apply Tomek Links
    tomek = TomekLinks()
    X_resampled, y_resampled = tomek.fit_resample(X, y)

    # Combine the resampled features and target into a new DataFrame
    df_resampled = pd.DataFrame(X_resampled, columns=X.columns)
    df_resampled['target'] = y_resampled
    
    return df_resampled


In [9]:
def apply_random_undersampling_and_scaling(df, random_state=42):
    
    # Drop 'target' column and separate features and target variable
    X = df.drop(['target'], axis='columns')  # Ensure the target column name is correct
    y = df['target']

    # Initialize the random undersampler
    rus = RandomUnderSampler(random_state=random_state)

    # Apply random undersampling
    X_resampled, y_resampled = rus.fit_resample(X, y)

    # Concatenate the resampled features and target into a new DataFrame
    resampled_df = pd.DataFrame(X_resampled, columns=X.columns)  # Create DataFrame from resampled features
    resampled_df['target'] = y_resampled  # Add the resampled target variable

    # Initialize the Min-Max scaler
    scaler = MinMaxScaler()

    # Apply Min-Max scaling to the feature columns only
    resampled_df[X.columns] = scaler.fit_transform(resampled_df[X.columns])

    return resampled_df


In [10]:
def xgboost_classifier(df, random_state=42):
    
    # Separate features and target variable
    X_train = df.drop(['target'], axis='columns')  # Features
    y_train = df['target']  # Target variable

    # Initialize the XGBoost classifier
    xgboost_model = XGBClassifier(
        n_estimators=100,
        learning_rate=0.1,
        random_state=random_state,
        use_label_encoder=False,
        eval_metric='logloss'
    )
    
    # Fit the model
    xgboost_model.fit(X_train, y_train)
    
    return xgboost_model

In [29]:
def evaluate_classification_model(y_true, y_pred):
    
    # Calculate confusion matrix and basic metrics
    cm = confusion_matrix(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    # Calculate True Negatives, False Positives, and False Negatives
    TN = cm[0, 0]  # True Negatives
    FP = cm[0, 1]  # False Positives
    FN = cm[1, 0]  # False Negatives
    TP = cm[1, 1]  # True Positives

    # Calculate Specificity (True Negative Rate)
    specificity = TN / (TN + FP) if (TN + FP) > 0 else 0

    # Calculate Negative Predictive Value (NPV)
    npv = TN / (TN + FN) if (TN + FN) > 0 else 0

    # Calculate Balanced Accuracy
    balanced_accuracy = (recall + specificity) / 2

    # Return evaluation metrics
    return {
        'Confusion Matrix': cm,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Specificity (TNR)': specificity,
        'Negative Predictive Value (NPV)': npv,
        'Balanced Accuracy': balanced_accuracy
    }

def plot_roc_curve(y_true, y_scores):
    if y_scores is not None:
        # Compute ROC curve and AUC
        fpr, tpr, _ = roc_curve(y_true, y_scores)
        roc_auc = auc(fpr, tpr)

        # Plot ROC Curve
        plt.figure()
        plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
        plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--')  # Diagonal line
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic')
        plt.legend(loc='lower right')
        plt.grid()
        plt.show()

        return roc_auc
    else:
        print("ROC cannot be plotted because y_scores is None.")
        return None

# Training

In [12]:
x_train = pd.read_csv(r"D:\project\GST_dataset\Train_60\Train_60\X_Train_Data_Input.csv")
y_train = pd.read_csv(r"D:\project\GST_dataset\Train_60\Train_60\Y_Train_Data_Target.csv")

In [14]:
df = pd.concat([x_train,y_train], axis='columns')

df = drop_columns(df)

cate_col, continue_col = categorize_columns(df)
    
# Step 2: Fill missing values with mode for certain columns
df = fillna_with_mode(df)  # You can specify other columns as needed

# Step 3: Impute missing values in specified columns with median
df = impute_missing_with_median(df, columns=['Column8', 'Column15', 'Column6'])

df =  combined_imputation(df, continue_col)

# Step 4: Remove outliers
df = remove_outliers(df)

# Step 5: Apply Tomek Links to balance the dataset
df_balanced = apply_tomek_links(df)

Estimating with: Ridge
Feature: Column1, Variance Difference: 0.0
Feature: Column2, Variance Difference: 0.0
Feature: Column3, Variance Difference: 0.15990083472145078
Feature: Column4, Variance Difference: 0.1573243987411318
Feature: Column5, Variance Difference: 0.21945992830953576
Feature: Column6, Variance Difference: 0.0
Feature: Column7, Variance Difference: 0.0
Feature: Column8, Variance Difference: 0.0
Feature: Column14, Variance Difference: 0.776537758681152
Feature: Column15, Variance Difference: 0.0
Feature: Column17, Variance Difference: 0.0
Feature: Column18, Variance Difference: 0.0


Estimating with: Lasso
Feature: Column1, Variance Difference: 0.0
Feature: Column2, Variance Difference: 0.0
Feature: Column3, Variance Difference: 0.12766155576819405
Feature: Column4, Variance Difference: 0.10632938469414388
Feature: Column5, Variance Difference: 0.21947585764160282
Feature: Column6, Variance Difference: 0.0
Feature: Column7, Variance Difference: 0.0
Feature: Column8, Vari

C:\Users\Rajvi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\impute\_iterative.py:801: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


Feature: Column1, Variance Difference: 0.0
Feature: Column2, Variance Difference: 0.0
Feature: Column3, Variance Difference: -0.15060212200548195
Feature: Column4, Variance Difference: -0.12914911092915116
Feature: Column5, Variance Difference: -0.0894743700472771
Feature: Column6, Variance Difference: 0.0
Feature: Column7, Variance Difference: 0.0
Feature: Column8, Variance Difference: 0.0
Feature: Column14, Variance Difference: -0.9999073305161021
Feature: Column15, Variance Difference: 0.0
Feature: Column17, Variance Difference: 0.0
Feature: Column18, Variance Difference: 0.0




In [15]:
# Step 6: Apply random undersampling and scaling
df_resampled = apply_random_undersampling_and_scaling(df_balanced, random_state=42)

# Step 7: Train the XGBoost model
xgboost_model = xgboost_classifier(df_resampled, random_state=42)


C:\Users\Rajvi\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [10:29:39] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [18]:
import joblib

# Step 8: Save the model
model_filename = 'D:/project/GST_dataset/xgboost_model.pkl'
joblib.dump(xgboost_model, model_filename)

print(f"Model saved as {model_filename}")

Model saved as D:/project/GST_dataset/xgboost_model.pkl


In [19]:
model_filename = 'D:/project/GST_dataset/xgboost_model.pkl'
xgboost_model = joblib.load(model_filename)

print("Model loaded successfully.")

Model loaded successfully.


In [22]:
x_test = pd.read_csv(r"D:\project\GST_dataset\Test_20\Test_20\X_Test_Data_Input.csv")
y_test = pd.read_csv(r"D:\project\GST_dataset\Test_20\Test_20\Y_Test_Data_Target.csv")

In [33]:
def test_model(xgboost_model, test_df):
    
    # Step 1: Drop unnecessary columns from test_df
    test_df = drop_columns(test_df)
    
    # Step 2: Categorize columns
    cate_col, continue_col = categorize_columns(test_df)
        
    # Step 3: Fill missing values with mode for certain columns
    test_df = fillna_with_mode(test_df)  # You can specify other columns as needed
    
    # Step 4: Impute missing values in specified columns with median
    test_df = impute_missing_with_median(test_df, columns=['Column8', 'Column15', 'Column6'])
    
    # Step 5: Combined imputation for continuous columns
    test_df = combined_imputation(test_df, continue_col)
    
    # Step 6: Remove outliers
    test_df = remove_outliers(test_df)
    
    X = test_df.drop(['target'], axis='columns')
    # Initialize the Min-Max scaler
    scaler = MinMaxScaler()

    # Apply Min-Max scaling to the feature columns only
    test_df[X.columns] = scaler.fit_transform(test_df[X.columns])

    # Step 9: Separate features and target variable
    X_test = test_df.drop(['target'], axis='columns')  # Features
    y_test = test_df['target']  # Target variable

    # Step 10: Predict using the trained model
    y_pred = xgboost_model.predict(X_test)

    # Step 11: Evaluate the model
    results = evaluate_classification_model(y_test, y_pred)

    print("\nXGBoost Model Evaluation Results:")
    print(results)

    y_scores = xgboost_model.predict_proba(X_test)[:, 1]
    roc_auc = plot_roc_curve(y_test, y_scores)

    return None


In [ ]:
df_test = pd.concat([x_test, y_test], axis='columns')

test_model(xgboost_model, df_test)

Estimating with: Ridge
Feature: Column1, Variance Difference: 0.0
Feature: Column2, Variance Difference: 0.0
Feature: Column3, Variance Difference: 0.1594936290178316
Feature: Column4, Variance Difference: 0.1503104017657545
Feature: Column5, Variance Difference: 0.30185586416404275
Feature: Column6, Variance Difference: 0.0
Feature: Column7, Variance Difference: 0.0
Feature: Column8, Variance Difference: 0.0
Feature: Column14, Variance Difference: 0.0002693680232042927
Feature: Column15, Variance Difference: 0.0
Feature: Column17, Variance Difference: 0.0
Feature: Column18, Variance Difference: 0.0


Estimating with: Lasso
Feature: Column1, Variance Difference: 0.0
Feature: Column2, Variance Difference: 0.0
Feature: Column3, Variance Difference: 0.12822914574213917
Feature: Column4, Variance Difference: 0.1067000990418191
Feature: Column5, Variance Difference: 0.3032276977800241
Feature: Column6, Variance Difference: 0.0
Feature: Column7, Variance Difference: 0.0
Feature: Column8, Var